# Lesson 4 - Creating a sinle WRF-Hydro model run through PEST++

The easiest way to check whether the coupling between the model and the PEST++ works as it should is to create a single model run. This single model run will later be used in Lesson 6 for weighting of the observations and building the objective function. To run PEST++ with WRF-Hydro, one would need to create a folder that has all the files from both systems plus pre and post-processing scripts for communicating the information between the two systems. You already got familiar with the required files for running WRF-Hydro model in Lesson 2, and we assume you have familiarity with the PEST++, we will quickly mention the files that are used in this training below. The pre and post processing scripts for running WRF-Hydro also has been provided here, and will be explained further below. 

In [ ]:
# load the required libraries 
import os 
import pyemu 
import pandas as pd

## **Step 1. Create a work directory and copy all the required information** 

This folder needs to have all the required information for running PEST++/WRF-Hydro which included the followings: 
* PEST++ binary files 
* PEST++ necessary files (control file, template file, instruction file) 
* WRF-Hydro executable and TBL files  
* WRF-Hydro example case (static files, forcing and namelists)
* WRF-Hydro pre and postprocessing scripts required to communicate with PEST++

In [ ]:
%%bash
# create the working directory
mkdir -p /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run

## **Step 2. Copy WRF-Hydro related files**

We will create a folder named `WRFHydro_Model` and copy the required WRF-Hydro model in there. 
We start with copying the WRF-Hydro executable and the TABLE files from the *~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run* directory. These files are small so we will make actual copies rather than symbolic links in this case. 

In [ ]:
%%bash

cd /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run

# create a new folder to hold all the WRF-Hydro files
mkdir WRFHydro_Model
cd WRFHydro_Model

# copy the wrf_hydro executable and the 
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL .
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe .

ls 

Next we need to copy all the specific domain information such as static input files (`DOMAIN`), the atmospheric forcing files (`FORCING`), the model restart files to have a warm start simulations (`RESTART`) and the namelits provided with the example case. We place all the required files for the WRFHydro execution in a folder called WRFHydro_Model. 

In [ ]:
%%bash 
# Let's copy the example case files
cd /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run/WRFHydro_Model
ln -sf ~/wrf-hydro-training/example_case/FORCING .
ln -sf ~/wrf-hydro-training/example_case/DOMAIN .
ln -sf ~/wrf-hydro-training/example_case/RESTARTS .

# copy namelist 
cp ~/wrf-hydro-training/example_case/namelist.hrldas .
cp ~/wrf-hydro-training/example_case/hydro.namelist .
ls 

## **Step 3. Copy PEST++ binary file**

To run PEST++, one would need to either compile PEST++ or use the available binary files provided with the code release. The binary files work on many systems. Here we have included the PEST++ binary files in the training material, and will copy it to the working directory. From the available binaries, we will use the `pestpp-ies` here, since we would like to perform the history matching using the iterative Ensemble Smoother (iES) technique. 


In [ ]:
%%bash 
# Move one directory up, to the working directory
cd /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run

# copy the PEST++ binary files 
cp /home/docker/wrf-hydro-training/pestpp_bin/pestpp-ies .

## **Step 4. Copy all WRF-Hydro/PESTPP files**

We have prepared the files that matches the example case provided here, and this includes both the files required by PEST++ (control file, instruction and template files) and the scripts for running the WRF-Hydro Pre-processing, execution and post processing.

In [ ]:
%%bash 
cd /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run

# copy all the pestpp required files 
cp /home/docker/wrf-hydro-training/example_case/PESTPP/* .
ls

let take a look at the content of the files provided. Programs of both the PEST and PEST++ suites require three types of input file. These are:
* template files, one for each model input file in which parameters or decision variables reside. Here `wrftpl.tpl` file is the template file prepared for WRF-Hydro. The PEST++ workflow will write the parameters in this file, and the `pre_processing_WRFHydro_run.py` file will read that file and apply the new set of parameters to the netcdf files. 
* instruction files, one for each model output file from which numbers must be read. Here `wrfpif.ins` file is the instruction file prepared for the WRF-Hydro. The `post_processing_WRFHydro_run.py` script should create a file with similar format and fill in the simulated streamflow values in place of entries such as `!obs_20180801_010000!`. 
* a control file, supplying the PEST or PEST++ program with the names of all template and instruction files, the names of corresponding model input and output files, the values of control variables, initial parameter values, measurement values, weights, etc. Here file `wrfpst.pst` file is the control file used for the training. 

In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run/wrftpl.tpl

In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run/wrfpif.ins | head -n 10

In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run/wrfpst.pst

### **calib_parameters.csv**: 
This file contains a subset of parameters that we could calibrate in WRF-Hydro. If one wish to calibrate any other parameters than what is provided in this table, 
we should make appropriate modifications to the `pre_processing_WRFHydro_run.py` script to make sure the parameter is changed in the appropriate WRF-Hydro file before running WRF-Hydro model. 

In [ ]:
calib_parameters = pd.read_csv("/home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run/calib_parameters.csv")
calib_parameters

`inputfile.csv` and `outputfile.csv` has the name of the files required by PEST++ and the WRF-Hydro for both parameters and the model observation/simulations. 

In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run/inputfile.csv
cat /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run/outputfile.csv

### How to run the model within PEST++ 
In the control file, there is an option `model command line` which user could specify the command that will run the model. Here we are using a shell script called `runCmd.sh` which does three things: 
* run the pre-processing script which takes the parameters from the `wrftpl_input.txt` file, and apply the parameters to the netcdf files. 
* run the WRF-Hydro model. 
* run the post-processing script which will write the streamflow simulation in a files called `wrfpif.txt` a mirror of the `wrfpif.ins` with the values. 


In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run/runCmd.sh

## **Step 5. Make modifications to the files**

Let s modify the namelists to read the netcdf files from the WRF-Hydro run directory instead of the DOMAIN directory, since we make modification to these files. The pre-processing script will read the parameters from the PEST++ output files and then modify the input netcdf files to WRF-Hydro. 

In [ ]:
%%bash 
cd /home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run/WRFHydro_Model/
# Edit namelist.hrldas 
sed -i -e 's#DOMAIN/soil_properties.nc#soil_properties.nc#g' namelist.hrldas
sed -i -e 's#DOMAIN/Fulldom.nc#Fulldom.nc#g' hydro.namelist
sed -i -e 's#DOMAIN/HYDRO_TBL_2D.nc#HYDRO_TBL_2D.nc#g' hydro.namelist
sed -i -e 's#DOMAIN/GWBUCKPARM.nc#GWBUCKPARM.nc#g' hydro.namelist

## **Step 6. Running PEST++/WRF-Hydro**

For the sake of this training, we will use a pyemy function called `utils.os_utils.start_workers` to run PEST++, which is suitable for training given the size of the experiment. Please refer to the PEST++ documentation for more information on this function (https://pyemu.readthedocs.io/en/develop/autoapi/pyemu/utils/os_utils/index.html). 


In [ ]:
#  run PEST++
pyemu.utils.os_utils.start_workers(worker_dir = "/home/docker/wrf-hydro-training/output/lesson4/Single_Model_Run", 
                                   exe_rel_path = "pestpp-ies", 
                                   pst_rel_path = "wrfpst.pst", 
                                   num_workers=1, 
                                   worker_root='/home/docker/wrf-hydro-training/output/lesson4/',
                                   master_dir = "/home/docker/wrf-hydro-training/output/lesson4/host",
                                   port=4004, 
                                   verbose = True, 
                                   cleanup = False)


## **Step 7. Let's check the outputs**

Let s check the host directory. 

In [ ]:
%%bash 
ls  /home/docker/wrf-hydro-training/output/lesson4/host

The successful finish message of PEST++ could be found in the `wrfpst.rmr` file. 

In [ ]:
%%bash 
cat  /home/docker/wrf-hydro-training/output/lesson4/host/wrfpst.rec | tail -n 5

Let us check the content of the workder folder. 

In [ ]:
%%bash 
ls  /home/docker/wrf-hydro-training/output/lesson4/worker_0/

Now we could continue to the next lesson where we submit a Monte Carlo model simulation using PEST++. 